In [24]:
import numpy as np 

arr = np.array([1,2,3,4,5])

p_arr = np.append(p_arr[0],p_arr)
q_arr = np.append(q_arr[0],q_arr)

arr

array([1, 1, 2, 3, 4, 5])

In [17]:
print(fx_arr2)

[0.125, 0.125, 0.125, 0.125, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [33]:
from tools import psi, dec_to_bin, bin_to_dec, full_encode
import os 
from fractions import Fraction

def get_phase_target(m):

    # define x array 
    n = 6
    x_min = 40
    x_max = 168 
    dx = (x_max-x_min)/(2**n) 
    x_arr = np.arange(x_min, x_max, dx) 

    # calculate target output for phase 
    phase_target = psi(np.linspace(0, 2**n, len(x_arr)))

    # calculate target for phase taking into account rounding 
    phase_reduced = np.modf(phase_target / (2* np.pi))[0] 
    phase_reduced_bin = [dec_to_bin(i,m, "unsigned mag", 0) for i in phase_reduced]
    phase_reduced_dec =  np.array([bin_to_dec(i,"unsigned mag", 0) for i in phase_reduced_bin])
    phase_rounded = 2 * np.pi * phase_reduced_dec

    return phase_rounded

def get_eps_chi(m,L,e,p,q):
    # assume "default conditions"

    file=os.path.join("outputs",f"weights_6_{m}(0)_{L}_{e}_psi_WILL_(S)(PR)(r)_{Fraction(p).numerator}-{Fraction(p).denominator}_{Fraction(q).numerator}-{Fraction(q).denominator}.npy") 
    alt_file=os.path.join("outputs",f"weights_6_{m}(0)_{L}_{e}_psi_WILL_(S)(PR)(r)_{p if p != 1 else int(p)}_{q}.npy")
    if os.path.isfile(file):
        weights_phase=file 
    elif os.path.isfile(alt_file):
        weights_phase=alt_file    
    else:
        raise FileNotFoundError(f"File '{file}' or '{alt_file}' not found.")
     
    weights_ampl = "ampl_outputs/weights_6_3_600_x76_MM_40_168_zeros.npy" 
    state_vec = full_encode(6,m, weights_ampl, weights_phase, 3, L,real_p=True,repeat_params=None)

    amplitude = np.abs(state_vec)
    phase = np.angle(state_vec) + 2* np.pi * (np.angle(state_vec) < -np.pi).astype(int)
    phase *= (amplitude > 1e-15).astype(float) 

    phase_rounded=get_phase_target(m)

    epsilon = 1 - np.sum(amplitude**2)
    chi = np.mean(np.abs(phase - phase_rounded))

    print(phase)
    print(phase_rounded)

    return epsilon, chi 

In [34]:
get_eps_chi(3,6,600,0.25,0.0)

[ 0.00188469  0.00294799  0.00198489  0.00295468  0.00113523  0.00094188
 -0.00037444 -0.00056129  0.01586584  0.01499351  0.01884041  0.01809011
  0.01253959  0.01044266  0.01399994  0.01214012  0.00223652  0.00342272
  0.00683936  0.00809252 -0.00030557 -0.00029668  0.0027156   0.00287576
  0.01754448  0.01705228  0.02571786  0.02550582  0.01253548  0.01091446
  0.01920368  0.01795829 -0.03153422 -0.01817619 -0.01891604 -0.0070412
 -0.02829049 -0.01624579 -0.01869577 -0.00802905 -0.03037316 -0.02024667
 -0.01564503 -0.00684589 -0.02940832 -0.02046521 -0.01752836 -0.00979103
 -0.01654178 -0.00420352  0.0011863   0.01228313 -0.01649838 -0.00541164
 -0.00188769  0.0080588  -0.01443214 -0.00514732  0.0061136   0.01433409
 -0.01646208 -0.00827849  0.00105173  0.00828312]
[0.78539816 0.78539816 0.78539816 0.78539816 0.78539816 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        

(0.03543503025018946, 0.0725858819365268)

In [ ]:
n = 6
    x_min = 40
    x_max = 168 
    dx = (x_max-x_min)/(2**n) 
    x_arr = np.arange(x_min, x_max, dx) 